In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
df_wiki = pd.read_csv('/content/drive/Shareddrives/ANLY 580 project /wiki_100000.csv')
df_wiki

,Unnamed: 0,id,url,title,text
0,0,1658014,https://en.wikipedia.org/wiki/Adrienne%20Mayor,Adrienne Mayor,Adrienne Mayor (born 1946) is a historian of a...
1,1,415109,https://en.wikipedia.org/wiki/Jo%20Stafford,Jo Stafford,"Jo Elizabeth Stafford (November 12, 1917July 1..."
2,2,733308,https://en.wikipedia.org/wiki/Milan%20Rapai%C4%87,Milan Rapaić,"Milan ""Miki"" Rapaić (born 16 August 1973) is a..."
3,3,2597099,https://en.wikipedia.org/wiki/Windsor%20North%...,Windsor North School,Windsor North School is a primary school in In...
4,4,690250,https://en.wikipedia.org/wiki/List%20of%20rive...,List of rivers of Missouri,List of rivers in Missouri (U.S. state).\n\nBy...
...,...,...,...,...,...
99995,99995,670737,https://en.wikipedia.org/wiki/Standesamt%20Ade...,Standesamt Adelnau,Standesamt Adelnau was one of the civil regist...
99996,99996,580501,https://en.wikipedia.org/wiki/Siaka%20Stevens,Siaka Stevens,Siaka Probyn Stevens (24 August 1905 – 29 May ...
99997,99997,2214203,https://en.wikipedia.org/wiki/Malitbog,Malitbog,Malitbog is the name of several places in the ...
99998,99998,2177759,https://en.wikipedia.org/wiki/Calyx%20%28music...,Calyx (musician),"Calyx is a British drum and bass act, speciali..."


In [4]:
df_sentence = pd.read_csv('/content/drive/Shareddrives/ANLY 580 project /wiki_sentence.csv')
#df_sentence = df_sentence.dropna()
df_sentence.head(10)

,sentence
0,adrienne mayor is a historian of ancient scien...
1,mayor specializes in ancient history and the s...
2,her work in pre scientific fossil discoveries ...
3,"mayor book, greek fire, poison arrows, the sco..."
4,"lifefrom number to number, she worked as a cop..."
5,"since number, mayor has been a research schola..."
6,mayor has published books and articles on the ...
7,her books the first fossil hunters and fossil ...
8,"feder book frauds, myths, and mysteries scienc..."
9,"her books have been translated into french, ge..."


In [5]:
len(df_sentence)

1174257

## Word Embedding Match

In [6]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 28.1 MB/s 
     |████████████████████████████████| 7.6 MB 58.6 MB/s 
     |████████████████████████████████| 182 kB 77.1 MB/s 


In [7]:
from sklearn.metrics.pairwise import cosine_similarity
import re
from tqdm import tqdm

class AutocompleteByEmbedding:

  def __init__(self, data, column, tokenizer, model):
    self.data = data
    self.col = column
    self.tokenizer = tokenizer
    self.model = model
  
  def match_prefix(self, sentence, prefix):
    matcher = r"\b" + prefix
    prefix_match = re.search(matcher, sentence)
    if prefix_match:
        return True
    else:
        return False
  
  def word_with_prefix(self, encoded, prefix):
    words = []
    for token in encoded['input_ids'][0]:
        words.append(self.tokenizer.decode([token]))
    index = []   
    for i in range(len(words)):
        if re.match(r'^' + prefix, words[i]):
            index.append(i)
    return index

  def autocomplete(self, sentence, document_size=200, top_n=5, similarity=True):
    words = sentence.split()
    prefix = words[-1]
    sentence_with_prefix = self.data[self.col].apply(lambda x: self.match_prefix(x, prefix))
    df_prefix = self.data[sentence_with_prefix]
    # tokenize input sentence
    encoded_input = self.tokenizer(text, return_tensors="pt")
    output = self.model(**encoded_input)
    # word embedding X
    input_embedding = output.last_hidden_state[-1].detach().numpy()
    word_similarity = {}
    # tokenize training sentence
    for document in tqdm(df_prefix[self.col][:document_size]):
        encoded_document = self.tokenizer(document, max_length=128, truncation=True, padding=True, return_tensors="pt")
        document_output = self.model(**encoded_document)
        Y = document_output.last_hidden_state[-1].detach().numpy()
        encoded_index = self.word_with_prefix(encoded_document, prefix)
        cos_distance = cosine_similarity(input_embedding, Y)
        for index in encoded_index:
            word = self.tokenizer.decode([encoded_document["input_ids"][0][index]])
            distance = cos_distance[len(input_embedding)-1-1, index]
            if word not in word_similarity:
                word_similarity[word] = distance
            elif word_similarity[word] < distance:
                word_similarity[word] = distance
            else:
                continue
    top_words = sorted(word_similarity, key=lambda x: -word_similarity[x])[:top_n]
    result = []
    for word in top_words:
        result.append((word, word_similarity[word]))
    if len(result) == 0:
        size = min(document_size, len(df_prefix))
        train_words = {}
        for document in df_prefix["sentence"][:size]:
            pattern = r"\b" + prefix + "[a-z]*"
            search = re.search(pattern, document)
            train_words[search[0]] = train_words.get(search[0], 0) + 1
        sorted_words = sorted(train_words, key=lambda x: -train_words[x])[:top_n]
        for word in sorted_words:
            result.append((word, train_words[word] / size))
    if len(result) == 0:
      print("Cannot complete word " + prefix)
    if similarity:
      return result
    else:
      predicted_words = []
      predicted_words = [pair[0] for pair in result]
      return predicted_words

In [8]:
from transformers import DistilBertTokenizer, DistilBertModel

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

bert_autocomplete = AutocompleteByEmbedding(data=df_sentence, column="sentence", tokenizer=tokenizer, model=model)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
text = "That which does not kill us make us str"
bert_autocomplete.autocomplete(text)

100%|██████████| 200/200 [00:27<00:00,  7.31it/s]


[('strangely', 0.5754252),
 ('strong', 0.5282924),
 ('street', 0.5196317),
 ('strategy', 0.4686629),
 ('structure', 0.44376093)]

In [ ]:
text = "Natural language processing is a subfield of linguistics, computer sci"
bert_autocomplete.autocomplete(text)

100%|██████████| 200/200 [00:33<00:00,  5.98it/s]


[('science', 0.6582532),
 ('sciences', 0.6273753),
 ('scientist', 0.53728306),
 ('scissors', 0.5202313),
 ('scientists', 0.51241314)]

In [ ]:
bert_autocomplete.autocomplete("t")

100%|██████████| 200/200 [00:23<00:00,  8.48it/s]


[('technology', 0.59731483),
 ('thought', 0.5147018),
 ('tactics', 0.51239157),
 ('terminology', 0.45969442),
 ('together', 0.4554178)]

In [ ]:
bert_autocomplete.autocomplete("p")

100%|██████████| 200/200 [00:28<00:00,  7.05it/s]


[('point', 0.42714956),
 ('performance', 0.42309967),
 ('peers', 0.4211117),
 ('perform', 0.41639417),
 ('poetic', 0.40952352)]

In [ ]:
# correct word: vrindavana
text = "kameshvara temple, in kamyavan, one of the twelve forests of vrind"
bert_autocomplete.autocomplete(text)

100%|██████████| 10/10 [00:01<00:00,  6.01it/s]


[('vrindavan', 0.4),
 ('vrindavana', 0.3),
 ('vrindavanam', 0.1),
 ('vrindabanin', 0.1),
 ('vrindaban', 0.1)]

In [ ]:
# correct word: subjunctive
text = "some important ones are declarative, affirmative, negative, emphatic, conditional, imperative, interrogative and subju"
bert_autocomplete.autocomplete(text)

100%|██████████| 88/88 [00:14<00:00,  6.24it/s]


[('subjugated', 0.4090909090909091),
 ('subjugation', 0.23863636363636365),
 ('subjugate', 0.1590909090909091),
 ('subjunctive', 0.09090909090909091),
 ('subjugating', 0.056818181818181816)]

In [9]:
text = "i have attached the file for your ref"
bert_autocomplete.autocomplete(text)

100%|██████████| 200/200 [00:41<00:00,  4.86it/s]


[('references', 0.5986207),
 ('refurbished', 0.52707314),
 ('reflections', 0.51532924),
 ('ref', 0.5110934),
 ('reference', 0.5101601)]

In [13]:
text = "jumping fox chasing c"
bert_autocomplete.autocomplete(text)

100%|██████████| 200/200 [00:30<00:00,  6.50it/s]


[('c', 0.8225794),
 ('company', 0.522678),
 ('club', 0.52197415),
 ('claims', 0.50594366),
 ('china', 0.50570184)]

## LSTM

In [ ]:
import numpy as np
import re
from keras.models import load_model
import pickle

In [ ]:
model = load_model("/content/drive/Shareddrives/ANLY 580 project /lstm/autocomplete_model.h5")
history = pickle.load(open("/content/drive/Shareddrives/ANLY 580 project /lstm/history.p", "rb"))

In [ ]:
class AutocompleteByLSTM:
  def __init__(self, model, set_of_chars, sequence_length, char_to_index, index_to_char):
    self.model = model
    self.set_of_chars = set_of_chars
    self.set_length = len(set_of_chars)
    self.sequence_length = sequence_length
    self.char_to_index = char_to_index
    self.index_to_char = index_to_char

  def encode_input(self, text):
    encoded_text = np.zeros((1, self.sequence_length, self.set_length))
    for t, char in enumerate(text):
      encoded_text[0, t, self.char_to_index[char]] = 1    
    return encoded_text

  def top_prediction(self, preds, n):
    preds = np.array(preds)
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return np.argsort(preds)[:len(preds)-n-1:-1]

  def predict_word(self, text):
    original_text = text
    completion = ''
    while True:
        x = self.encode_input(text)
        preds = self.model.predict(x, verbose=0)[0]
        next_index = self.top_prediction(preds, n=1)[0]
        next_char = self.index_to_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        if (next_char == ' ') or (completion[-1:] == completion[-2:-1]) or (len(completion) > 15):
            return completion
  
  def predict_words_n(self, text, n):
    encoded_text = self.encode_input(text)
    preds = self.model.predict(encoded_text, verbose=0)[0]
    next_indices = self.top_prediction(preds, n)
    return [self.index_to_char[index] + self.predict_word(text[1:] + self.index_to_char[index]) for index in next_indices]

  def autocomplete(self, sentence, n=3):
    if len(sentence) < self.sequence_length:
      print(sentence)
      print(f"Please provde at least {self.sequence_length} characters")
    else:
      seq = sentence[len(sentence)-self.sequence_length:]
      predictions = self.predict_words_n(seq.lower(), n)
      print(sentence)
      print(predictions)
      return predictions  

In [ ]:
set_of_chars = sorted([' ', '!', ',', '.', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j',
  'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'])
char_to_index = dict((c, i) for i, c in enumerate(set_of_chars))
index_to_char = dict((i, c) for i, c in enumerate(set_of_chars))
sequence_length = 32
word_completion = AutocompleteByLSTM(model, set_of_chars, sequence_length, char_to_index, index_to_char)

In [ ]:
sentence_example = ["This is an incomplete example se", 
           "Sensitivity refers to the probab", 
           "Natural language processing is a subfield of linguistics, computer sci", 
           "Mayor specializes in ancient history",
           "Since number, mayor has been a research",
           "Sentence is a set of words that is comp",
           "Machine learning is a field of inquiry devoted to und"]
for sentence in sentence_example:
    word_completion.autocomplete(sentence, 5)

This is an incomplete example se
['rved ', 'cret ', 'en ', 'ason ', 't ']
Sensitivity refers to the probab
['ly ', ' and ', 'ily ', 'e ', 'ory ']
Natural language processing is a subfield of linguistics, computer sci
['ence ', 'le ', 'te ', 'nce ', 're ']
Mayor specializes in ancient history
[' of ', ', ', '. ', 'ing ', 's ']
Since number, mayor has been a research
[' and ', 'er ', ', ', 'ing ', '. ']
Sentence is a set of words that is comp
['any ', 'leted ', 'eted ', 'osed ', 'rised ']
Machine learning is a field of inquiry devoted to und
['er ', 'ive ', 'ar ', 'ourgh ', 'ust ']
